In [1]:
# use the cell below to ensure correct relative imports
# you must be in the qcore directory

In [2]:
cd ../qcore

C:\Users\qcrew\Desktop\qcore\qcore


### Proto-user guide for interacting with the Stage object in qcore

Motivation: a way to control all Instruments that you're interacting with as part of an experiment.

Definition: an experimental stage is a group of instruments needed to run a given experiment. These instruments may be physical hardware or may be meta-instruments (e.g. qubit, rr)

1. With Stage, you can initialize all your instruments from a single yaml file. Initialize means the instrument is connected, set with initial parameters, and ready for use!
2. You can add (`enter`) or remove (`exit`) instruments from the stage. You can also `enter` instruments that are saved in yaml files!
3. You can get current state of all the instruments in the stage using `stage.parameters` !!!

## Goal - we will build up the experimental stage that we're currently using to characterize a 1 qubit, 1 rr device with opx, sa, labbricks

### After we do so, we'll save it as a yaml file so that the next time we can simply load it from that yaml file and we'll be good to go!!!

Note - as of the writing of this tutorial, the qm is not integrated into qcore yet (but there's a qm_config_builder to automate the qm config building step), and the mixer tuning step has not yet been developed and integrated with the rest of the codebase.

In [3]:
from pathlib import Path

from instruments import LabBrick, qm_config_builder, QuantumDevice, Sa124, Stage

In [4]:
# the stage to be built up
stage = Stage()

#### Step 1 - Add the QuantumDevice

In [5]:
# we have already created a quantum device in the cqed_components tutorial
# we saved the device in 'example_device.yaml' in this folder
# let's create a variable that remembers this path

device_yaml_path = Path.cwd().parent / 'tutorials/example_device.yaml'

In [6]:
# let's add the device to the stage by calling enter()
stage.enter(path=device_yaml_path)

In [7]:
# in 'example_device.yaml', our device is named 'device_A'
# so we expect stage to have device_A as an attribute, let's see if it does
stage.device_A

In [8]:
# let's check what elements are part of device_A
stage.device_A.elements

{'qubit': <instruments.meta.cqed_components.QuantumElement at 0x1b20774e2e0>,
 'rr': <instruments.meta.cqed_components.QuantumElement at 0x1b208034610>}

In [9]:
# indeed, we see that the device has been added to the stage
# we can double confirm this by running
stage.parameters

{'device_A': {'qubit': {'lo_freq': 5000000000.0,
   'int_freq': 50000000.0,
   'ports': {'I': 1, 'Q': 2},
   'operations': {'CW': <utils.pulselib.Pulse at 0x1b208038e50>,
    'gaussian': <utils.pulselib.Pulse at 0x1b208038e80>}},
  'rr': {'lo_freq': 8000000000.0,
   'int_freq': -50000000.0,
   'ports': {'I': 3, 'Q': 4, 'out': 1},
   'operations': {'CW': <utils.pulselib.Pulse at 0x1b208038e50>,
    'readout': <utils.pulselib.MeasurementPulse at 0x1b208038b80>},
   'time_of_flight': 824,
   'smearing': 0}}}

#### Step 2 - Add the labbricks

Note: refer to the labbrick tutorial for more info on interacting with labbricks

In [10]:
# first, we need to know what frequency to set each labbrick at
# for that, we'll need to access the lo_freqs of both qubit and rr in our QuantumDevice
# as we're dealing with meta instruments, we can simply chain attributes to achieve this!
qubit_lo_freq = stage.device_A.qubit.lo_freq
rr_lo_freq = stage.device_A.rr.lo_freq

print('Qubit LO freq is ' + '{:7E}'.format(qubit_lo_freq) + ' Hz')
print('RR LO freq is ' + '{:7E}'.format(rr_lo_freq) + ' Hz')

Qubit LO freq is 5.000000E+09 Hz
RR LO freq is 8.000000E+09 Hz


In [11]:
# create the qubit labbrick
# verify that freq range of labbrick is 4-8GHz
lb_qubit = LabBrick(name='lb_qubit', serial_number=25331, frequency=qubit_lo_freq, power=15)

Trying to initialize lb_qubit
Connnected to LabBrick 25331
Setting initial parameters...
Successfully set frequency to 5.0000000E+09
Successfully set power to +15
LabBrick is ready to use.


In [12]:
# create the rr labbrick
# verify that freq range of labbrick is 5-10GHz
lb_rr = LabBrick(name='lb_rr', serial_number=25335, frequency=rr_lo_freq, power=13)

Trying to initialize lb_rr
Connnected to LabBrick 25335
Setting initial parameters...
Successfully set frequency to 8.0000000E+09
Successfully set power to +13
LabBrick is ready to use.


In [13]:
# let's add the two labbricks to stage
# to add multiple instruments, you pass them in as a Python set {lb_qubit, lb_rr}
stage.enter(instruments={lb_qubit, lb_rr})

In [14]:
# let's verify that the LabBricks have indeed been added
stage.parameters

# remember that we can access the labbricks with
#stage.lb_qubit
#stage.lb_rr

{'device_A': {'qubit': {'lo_freq': 5000000000.0,
   'int_freq': 50000000.0,
   'ports': {'I': 1, 'Q': 2},
   'operations': {'CW': <utils.pulselib.Pulse at 0x1b208038e50>,
    'gaussian': <utils.pulselib.Pulse at 0x1b208038e80>}},
  'rr': {'lo_freq': 8000000000.0,
   'int_freq': -50000000.0,
   'ports': {'I': 3, 'Q': 4, 'out': 1},
   'operations': {'CW': <utils.pulselib.Pulse at 0x1b208038e50>,
    'readout': <utils.pulselib.MeasurementPulse at 0x1b208038b80>},
   'time_of_flight': 824,
   'smearing': 0}},
 'lb_qubit': {'frequency': '5.0000000E+09', 'power': 15},
 'lb_rr': {'frequency': '8.0000000E+09', 'power': 13}}

#### Step 3 - Add the SA124

Note: refer to the sa124 tutorial for more info on interacting with sa124

In [15]:
# let's create the sa
# we'll set the center of the sweep at the rr_lo_freq
sa = Sa124(name='sa', serial_number=19184645, center=rr_lo_freq, span=500e6)

Trying to initialize sa, will take about 5s...
Connnected to SA124B 19184645
Configured sweep! Sweep info: 
{'start': '7.7501000E+09', 'center': '8.0000000E+09', 'span': '5.000E+08', 'sweep_length': 2500, 'rbw': '2.500E+05', 'ref_power': 0, 'bin_size': '2.000E+05'}


In [16]:
# let's add the sa to stage
# notice that to add a single instrument, you use the argument name 'instrument'
stage.enter(instrument=sa)

In [17]:
# verify that the sa has been added
stage.parameters

# the sa can be accessed as
#stage.sa

{'device_A': {'qubit': {'lo_freq': 5000000000.0,
   'int_freq': 50000000.0,
   'ports': {'I': 1, 'Q': 2},
   'operations': {'CW': <utils.pulselib.Pulse at 0x1b208038e50>,
    'gaussian': <utils.pulselib.Pulse at 0x1b208038e80>}},
  'rr': {'lo_freq': 8000000000.0,
   'int_freq': -50000000.0,
   'ports': {'I': 3, 'Q': 4, 'out': 1},
   'operations': {'CW': <utils.pulselib.Pulse at 0x1b208038e50>,
    'readout': <utils.pulselib.MeasurementPulse at 0x1b208038b80>},
   'time_of_flight': 824,
   'smearing': 0}},
 'lb_qubit': {'frequency': '5.0000000E+09', 'power': 15},
 'lb_rr': {'frequency': '8.0000000E+09', 'power': 13},
 'sa': {'start': '7.7501000E+09',
  'center': '8.0000000E+09',
  'span': '5.000E+08',
  'sweep_length': 2500,
  'rbw': '2.500E+05',
  'ref_power': 0,
  'bin_size': '2.000E+05'}}

#### Step 4 - Save the stage in yaml file

Now, we have built up our stage, time to save it!!!

In [18]:
# we first create the Path object that corresponds to the location we want to save the stage in
stage_yaml_file_name = 'example_stage.yaml'
stage_yaml_file_path = Path.cwd().parent / 'tutorials' / stage_yaml_file_name

In [19]:
# we call save()
stage.save(stage_yaml_file_path)
# and we should be good to go to the next step!

#### Step 5 - Load the saved stage and proceed to conduct experiments like a boss!

Note: as we'll be loading the same instruments as above, we'll first clear out the existing stage and safely disconnect from the physical instruments on that stage...

In [20]:
# simply call exit() with the flag exit_all=True to clear the whole stage
# any physical instruments on the stage will be disconnected
# exiting also deletes attributes of the Stage object, so you will no longer have attribute access to the instruments after exit
stage.exit(exit_all=True)

lb_qubit disconnected!
lb_rr disconnected!
sa disconnected!


In [21]:
# to load a stage from a yaml file, we call its classmethod load()
new_stage = Stage.load(stage_yaml_file_path)

Trying to initialize lb_qubit
Connnected to LabBrick 25331
Setting initial parameters...
Successfully set frequency to 5.0000000E+09
Successfully set power to +15
LabBrick is ready to use.
Trying to initialize lb_rr
Connnected to LabBrick 25335
Setting initial parameters...
Successfully set frequency to 8.0000000E+09
Successfully set power to +13
LabBrick is ready to use.
Trying to initialize sa, will take about 5s...
Connnected to SA124B 19184645
Configured sweep! Sweep info: 
{'start': '7.7501000E+09', 'center': '8.0000000E+09', 'span': '5.000E+08', 'sweep_length': 2500, 'rbw': '2.500E+05', 'ref_power': 0, 'bin_size': '2.000E+05'}


In [22]:
# let's map the instrument objects to stage attributes for easy access
device = new_stage.device_A
qubit = new_stage.device_A.qubit
rr = new_stage.device_A.rr
lb_qubit = new_stage.lb_qubit
lb_rr = new_stage.lb_rr
sa = new_stage.sa

In [23]:
# now let's use the qm config builder to build the qm config from our QuantumElements
# the build_qm_config() method accepts a Python set of elements
element_set = {qubit, rr}
config = qm_config_builder.build_qm_config(element_set)
print(config)

{'version': 1, 'elements': {'rr': {'intermediate_frequency': -50000000.0, 'smearing': 0, 'time_of_flight': 824, 'mixInputs': {'I': ('con1', 3), 'Q': ('con1', 4), 'lo_frequency': 8000000000.0, 'mixer': 'mixer_rr'}, 'operations': {'CW': 'CW_pulse', 'readout': 'readout_pulse'}, 'outputs': {'out1': ('con1', 1)}}, 'qubit': {'intermediate_frequency': 50000000.0, 'mixInputs': {'I': ('con1', 1), 'Q': ('con1', 2), 'lo_frequency': 5000000000.0, 'mixer': 'mixer_qubit'}, 'operations': {'CW': 'CW_pulse', 'gaussian': 'gaussian_pulse'}}}, 'controllers': {'con1': {'analog_outputs': {3: {'offset': 0.0}, 4: {'offset': 0.0}, 1: {'offset': 0.0}, 2: {'offset': 0.0}}, 'analog_inputs': {1: {'offset': 0.0}}, 'digital_outputs': {}, 'type': 'opx1'}}, 'mixers': {'mixer_rr': [{'intermediate_frequency': -50000000.0, 'correction': (1.0, 0.0, 0.0, 1.0), 'lo_frequency': 8000000000.0}], 'mixer_qubit': [{'intermediate_frequency': 50000000.0, 'correction': (1.0, 0.0, 0.0, 1.0), 'lo_frequency': 5000000000.0}]}, 'integrat

In [24]:
# let's open a new qm using the QMM and QM API
from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.qua import *

qmm = QuantumMachinesManager()
qm = qmm.open_qm(config)

2021-04-20 20:50:55,523 - qm - INFO - Performing health check
2021-04-20 20:50:55,523 - qm - INFO - Health check passed


In [25]:
# let's run a simple qua program to see if things are working...
with program() as cw:
    with infinite_loop_():
        play('CW', 'qubit')
        play('CW', 'rr')
job = qm.execute(cw)

2021-04-20 20:50:59,506 - qm - INFO - Flags: 
2021-04-20 20:50:59,506 - qm - INFO - Executing high level program


In [26]:
job.halt()

True